### Plot data from the netCDF conversions of the original log files
This Notebook will let you make time series plots of any of the variables from any of the intruments from the processed data available on your computer. Before running you need to first execute the `logs2netcdfs.py` program to download and process data from the rawdata archive; use the `-v` option to monitor its progress, otherwise it works silently.

To run this Notebook follow instructions at https://github.com/mbari-org/auv-python.

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/data'))
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import hvplot.pandas
import ipywidgets as widgets
from logs2netcdfs import BASE_PATH
from logs2netcdfs import LOG_FILES

# Pick auv_name available from locally processed files
try:
    auv_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(BASE_PATH))
                                                    if f not in ['.DS_Store', '.pytest_cache']],
                                                    description='AUV Name:')
    display(auv_picker)
except FileNotFoundError:
    print(f"No locally processed missions found in {BASE_PATH} -- cannot continue.")
    print("Execute 'python logs2netcdfs.py' to process mission files.")

In [ ]:
# Pick mission available for the auv_name
auv_name = auv_picker.value
missions_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs')
mission_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(missions_dir)) 
                                                   if f != '.DS_Store'], description='Mission:')
display(mission_picker)

In [ ]:
# Read all the netCDF file data into in-memory xarray objects saved in a dictionary
netcdfs_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs', mission_picker.value)
ncs = {}
for lf in LOG_FILES:
    nc_file = lf.replace('.log', '.nc')
    try:
        ncs[nc_file] = xr.open_dataset(os.path.join(netcdfs_dir, nc_file), decode_timedelta=False)
    except FileNotFoundError:
        pass
    except ValueError as e:
        print('Could not open netCDF file {}: {}'.format(nc_file, e))

In [ ]:
# Show all the attributes of each data set and all the variables in each data set
for nc_file, ds in ncs.items():
    print(f"{nc_file}: {[v for v in list(ds)]}")
    ##display(ds) # Uncomment to explore all the attributes

In [ ]:
# Pick the converted log to examine
ds_picker = widgets.Dropdown(options=ncs.keys(), description='File:')
display(ds_picker)

In [ ]:
# Pick the variable to plot
var_picker = widgets.Dropdown(options=list(ncs[ds_picker.value]), description='Variable')
display(var_picker)

In [ ]:
# Make an interactive plot
ds = ncs[ds_picker.value]
variable = var_picker.value
    
var = getattr(ds, variable)
yl = f"{var_picker.value} from {var.attrs['long_name']} [{var.attrs.get('units', '')}]"
print(f"{auv_name}: {mission_picker.value}: {ds_picker.value}: {var_picker.value}")
display(var.to_pandas().hvplot(ylabel=yl))